# 전국 지진옥외대피소 데이터
: 전국에 있는 지진옥외대피소에 대한 정보

출처
    - 행정안전부 https://www.data.go.kr/data/15039250/openapi.do

처리목적
    - 전국에 있는 지진옥외대피소의 위치와 이름 확인

처리사항

    1. 전국 지진옥외대피소 데이터 살펴보고 위도와 경도가 이상한 데이터 수정 
    2. 결과 시각화

In [1]:
import pandas as pd
import numpy as np

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


#### 데이터 불러오기

In [3]:
e_s = pd.read_csv('/content/drive/MyDrive/아/데이터/전국 지진옥외대피소.csv')
# e_s = pd.read_csv('/Users/jini/Documents/workspace/주간프로젝트/전국 지진옥외대피소.csv')
e_s.head(3)

,Unnamed: 0,ctprvn_nm,sgg_nm,vt_acmdfclty_nm,rdnmadr_cd,bdong_cd,hdong_cd,dtl_adres,fclty_ar,xcord,ycord
0,0,인천광역시,부평구,부들공원,NaN,2823710500,2823710500,인천광역시 부평구 삼산동 541,6100,126.74910532404219,37.5157272038413
1,1,인천광역시,부평구,시냇물공원,NaN,2823710500,2823710500,인천광역시 부평구 삼산동 443-1,640,126.736539,37.513312
2,2,인천광역시,부평구,가족공원(삼산2-4공원),NaN,2823710500,2823710500,인천광역시 부평구 삼산동 389-7,803,126.74316536374003,37.52116564781989


#### 데이터 살펴보기

In [4]:
e_s.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11000 entries, 0 to 10999
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   Unnamed: 0       11000 non-null  int64 
 1   ctprvn_nm        11000 non-null  object
 2   sgg_nm           11000 non-null  object
 3   vt_acmdfclty_nm  11000 non-null  object
 4   rdnmadr_cd       7971 non-null   object
 5   bdong_cd         11000 non-null  int64 
 6   hdong_cd         11000 non-null  int64 
 7   dtl_adres        11000 non-null  object
 8   fclty_ar         11000 non-null  int64 
 9   xcord            11000 non-null  object
 10  ycord            11000 non-null  object
dtypes: int64(4), object(7)
memory usage: 945.4+ KB


In [5]:
# 결측치 확인
e_s.isna().sum() # 필요한 대피소이름, x/y 좌표 컬럼에는 결측치 없음

Unnamed: 0            0
ctprvn_nm             0
sgg_nm                0
vt_acmdfclty_nm       0
rdnmadr_cd         3029
bdong_cd              0
hdong_cd              0
dtl_adres             0
fclty_ar              0
xcord                 0
ycord                 0
dtype: int64

In [6]:
# 중복값 확인
print(e_s.duplicated(keep=False).sum())

0


## MarkerCluster를 적용한 시각화

In [7]:
import folium

#### 지도시각화를 위한 데이터프레임 생성

In [8]:
e_s.drop(['Unnamed: 0', 'ctprvn_nm', 'sgg_nm', 'rdnmadr_cd', 'bdong_cd', 'hdong_cd',
          'dtl_adres', 'fclty_ar'], axis=1, inplace=True)
e_s.head()

,vt_acmdfclty_nm,xcord,ycord
0,부들공원,126.74910532404219,37.5157272038413
1,시냇물공원,126.736539,37.513312
2,가족공원(삼산2-4공원),126.74316536374003,37.52116564781989
3,동심공원(삼산1-4공원),126.74221830218373,37.50948972864331
4,나래공원(삼산2-7공원),126.74576737281392,37.51902424573734


In [9]:
# 이상한 데이터 수정(x좌표에 y좌표까지 같이 들어가 있음)
e_s[e_s['xcord'].str.contains('35.481726, 126.481198')]

,vt_acmdfclty_nm,xcord,ycord
5279,미산마을광장,"35.481726, 126.481198",35.481726


In [10]:
e_s.loc[5297, 'xcord'] = '35.481726'
e_s.loc[5297, 'ycord'] = '126.481198'

In [11]:
e_s.loc[5279, 'xcord'] = '35.481726'
e_s.loc[5279, 'ycord'] = '35.481726'

In [12]:
e_s[e_s['xcord'].str.contains('128.37.08')]

,vt_acmdfclty_nm,xcord,ycord
6598,팔룡어울림운동장,128.37.08,35.15.02


In [13]:
e_s.loc[6598, 'xcord'] = '128.3708'
e_s.loc[6598, 'ycord'] = '35.1502'

In [14]:
e_s[e_s['xcord'].str.contains('128.36.28')]

,vt_acmdfclty_nm,xcord,ycord
6599,팔룡미관공원,128.36.28,35.15.14


In [15]:
e_s.loc[6599, 'xcord'] = '128.3628'
e_s.loc[6599, 'ycord'] = '35.1514'

#### MarkerCluster 추가

In [16]:
from folium.plugins import MarkerCluster

In [ ]:
m = folium.Map(location=[37.57402594173796, 126.97385569402873],
              zoom_start=15)


marker_cluster = MarkerCluster().add_to(m)

for lat, long, name in zip(e_s['ycord'], e_s['xcord'], e_s['vt_acmdfclty_nm']):
    folium.Marker([lat, long], icon=folium.Icon(color='green'),
                 popup=folium.Popup(name, max_width=200)).add_to(marker_cluster)
    
m

In [ ]:
# 지도 위치에 안 맞는 데이터 수정(위치가 북반구, 터키, 일본에 찍힌 데이터)
# 계룡중학교(34.88560916468874, 128.62804098247773), 신도림초등학교 운동장(37.51225537859192, 126.88249426832837),
# 미산마을광장(35.48176095191277, 126.48114774793488), 봉계초등학교(36.15667645804526, 128.05412629147295)
e_s[e_s['vt_acmdfclty_nm'].str.contains('계룡중학교')]

e_s.loc[5297, 'xcord'] = '34.88560916468874'
e_s.loc[5297, 'ycord'] = '128.62804098247773'

In [ ]:
e_s[e_s['vt_acmdfclty_nm'].str.contains('신도림초등학교 운동장')]

e_s.loc[1609, 'xcord'] = '37.51225537859192'
e_s.loc[1609, 'ycord'] = '126.88249426832837'

In [ ]:
e_s[e_s['vt_acmdfclty_nm'].str.contains('미산마을광장')]

e_s.loc[5279, 'xcord'] = '35.48176095191277'
e_s.loc[5279, 'ycord'] = '126.48114774793488'

In [ ]:
e_s[e_s['vt_acmdfclty_nm'].str.contains('봉계초등학교')]

e_s.loc[4657, 'xcord'] = '36.15667645804526'
e_s.loc[4657, 'ycord'] = '128.05412629147295'

In [ ]:
# 지도 다시
m = folium.Map(location=[37.57402594173796, 126.97385569402873],
              zoom_start=15)


marker_cluster = MarkerCluster().add_to(m)

for lat, long, name in zip(e_s['ycord'], e_s['xcord'], e_s['vt_acmdfclty_nm']):
    folium.Marker([lat, long], icon=folium.Icon(color='green'),
                 popup=folium.Popup(name, max_width=200)).add_to(marker_cluster)
    
m